In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rfm-uas/UAS_RFM_csv.csv
/kaggle/input/uas-rfm/UAS_RFM_csv.csv


In [57]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [58]:
dataTD = pd.read_csv('../input/uas-rfm/UAS_RFM_csv.csv', sep=';', parse_dates=['TrxDate'])
dataTD.head(10)

,CardID,TrxDate,Amount
0,1000152,2021-10-06,844937
1,1000158,2021-06-25,385023
2,1000156,2021-04-03,691676
3,1000115,2021-12-02,571656
4,1000164,2021-03-26,107267
5,1000136,2021-09-02,447471
6,1000147,2021-04-22,358221
7,1000118,2021-08-05,252721
8,1000167,2021-03-29,376027
9,1000160,2021-04-01,802956


In [59]:
dataTD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 KB


In [60]:
#Identifikasi tanggal transaksi paling awal dan terbaru.
print(dataTD['TrxDate'].min(), dataTD['TrxDate'].max())

2021-01-01 00:00:00 2021-12-06 00:00:00


In [61]:
sd = dt.datetime(2021,12,7)
dataTD['hist']=sd - dataTD['TrxDate']
dataTD['hist'].astype('timedelta64[D]')
dataTD['hist']=dataTD['hist'] / np.timedelta64(1, 'D')
dataTD.head(10)

,CardID,TrxDate,Amount,hist
0,1000152,2021-10-06,844937,62.0
1,1000158,2021-06-25,385023,165.0
2,1000156,2021-04-03,691676,248.0
3,1000115,2021-12-02,571656,5.0
4,1000164,2021-03-26,107267,256.0
5,1000136,2021-09-02,447471,96.0
6,1000147,2021-04-22,358221,229.0
7,1000118,2021-08-05,252721,124.0
8,1000167,2021-03-29,376027,253.0
9,1000160,2021-04-01,802956,250.0


In [62]:
dataTD=dataTD[dataTD['hist'] < 250]
dataTD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 0 to 198
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   118 non-null    int64         
 1   TrxDate  118 non-null    datetime64[ns]
 2   Amount   118 non-null    int64         
 3   hist     118 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 4.6 KB


In [63]:
rfmTable = dataTD.groupby('CardID').agg({'hist': lambda x:x.min(), # Recency
                                        'CardID': lambda x: len(x), # Frequency
                                        'Amount': lambda x: x.sum()}) # Monetary Value

rfmTable.rename(columns={'hist': 'recency', 
                         'CardID': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)
rfmTable.head(10)

,recency,frequency,monetary_value
CardID,,,
1000101,204.0,1,375060
1000102,162.0,3,2443830
1000103,171.0,1,536315
1000104,167.0,1,455561
1000105,160.0,3,2291296
1000106,205.0,2,1357938
1000107,62.0,1,838617
1000108,165.0,1,266902
1000109,157.0,1,612271


In [64]:
quartiles = rfmTable.quantile(q=[0.25,0.30,0.35])
print(quartiles, type(quartiles))

      recency  frequency  monetary_value
0.25     62.0        1.0        455561.0
0.30     64.0        1.0        541481.0
0.35     96.0        1.0        605174.0 <class 'pandas.core.frame.DataFrame'>


In [65]:
#untuk recency
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.30]:
        return 3
    elif x <= d[p][0.35]: 
        return 2
    else:
        return 1
    
#untuk frequency dan monetary
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.30]:
        return 2
    elif x <= d[p][0.35]: 
        return 3
    else:
        return 4 

In [66]:
rfmSeg = rfmTable
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quartiles,))
rfmSeg['M_Quartile'] = rfmSeg['monetary_value'].apply(FMClass, args=('monetary_value',quartiles,))

In [67]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)
rfmSeg.head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CardID,,,,,,,
1000101,204.0,1,375060,1,1,1,111
1000102,162.0,3,2443830,1,4,4,144
1000103,171.0,1,536315,1,1,2,112
1000104,167.0,1,455561,1,1,1,111
1000105,160.0,3,2291296,1,4,4,144
1000106,205.0,2,1357938,1,4,4,144
1000107,62.0,1,838617,4,1,4,414
1000108,165.0,1,266902,1,1,1,111
1000109,157.0,1,612271,1,1,4,114


In [68]:
rfmSeg['Total_Class'] = rfmSeg['R_Quartile'] + rfmSeg['F_Quartile'] + \
rfmSeg['M_Quartile']


rfmSeg.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,Total_Class
CardID,,,,,,,,
1000101,204.0,1,375060,1,1,1,111,3
1000102,162.0,3,2443830,1,4,4,144,9
1000103,171.0,1,536315,1,1,2,112,4
1000104,167.0,1,455561,1,1,1,111,3
1000105,160.0,3,2291296,1,4,4,144,9


In [69]:
print("Regular Customer: ",len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Customer: ',len(rfmSeg[rfmSeg['F_Quartile']==4]))
print("Good Customer: ",len(rfmSeg[rfmSeg['M_Quartile']==4]))
print("Risk: ",len(rfmSeg[rfmSeg['R_Quartile']==1]))
print('Customer Gone: ', len(rfmSeg[rfmSeg['RFMClass']=='111']))
print('Customer Potential: ', len(rfmSeg[rfmSeg['RFMClass']=='333']) + len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Help: ', len(rfmSeg[rfmSeg['RFMClass']=='222']) + len(rfmSeg[rfmSeg['RFMClass']=='111']))

Regular Customer:  10
Customer:  31
Good Customer:  39
Risk:  38
Customer Gone:  8
Customer Potential:  10
Help:  8
